```
This software is part of GPU Ocean. 

Copyright (C) 2022 SINTEF Digital

Stress test to see how two simulators compete to use the same textures

This program is free software: you can redistribute it and/or modify
it under the terms of the GNU General Public License as published by
the Free Software Foundation, either version 3 of the License, or
(at your option) any later version.

This program is distributed in the hope that it will be useful,
but WITHOUT ANY WARRANTY; without even the implied warranty of
MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE.  See the
GNU General Public License for more details.

You should have received a copy of the GNU General Public License
along with this program.  If not, see <http://www.gnu.org/licenses/>.
```

# Simple minimal script for running a real-world simulation 

In [ ]:
#Lets have matplotlib "inline"
%matplotlib inline

import os
import sys

#Import packages we need
import numpy as np
import datetime
from netCDF4 import Dataset
from IPython.display import display

#For plotting
import matplotlib
from matplotlib import pyplot as plt
import matplotlib as mpl
from matplotlib import animation, rc
plt.rcParams["animation.html"] = "jshtml"
mpl.rcParams['figure.facecolor'] = 'white'

In [ ]:
from gpuocean.SWEsimulators import CDKLM16
from gpuocean.utils import Common, IPythonMagic, NetCDFInitialization, WindStress


In [ ]:
%cuda_context_handler gpu_ctx1

In [ ]:
%cuda_context_handler gpu_ctx2

In [ ]:
print(gpu_ctx1)
print(gpu_ctx2)

### Plotting functionality

In [ ]:
def plotSolution(fig, sim, comment="",
                 eta_max=2, hu_max=50, 
                 ax=None, sp=None):

    eta, hu, hv = sim.download(interior_domain_only=True)
    dx, dy, t = sim.dx, sim.dy, sim.t
    
    from datetime import timedelta
    fig.suptitle("Time = {:0>8} ({:s})".format(str(timedelta(seconds=int(t))), comment), 
                 fontsize=18,
                 horizontalalignment='left')
    
    ny, nx = eta.shape
    domain_extent = [0, nx*dx, 0, ny*dy]
   
    if (ax is None):
        ax = [None]*3
        sp = [None]*3
        
        ax[0] = plt.subplot(1, 3, 1)
        sp[0] = ax[0].imshow(eta, interpolation="none", origin='lower', 
                             cmap=plt.cm.BrBG, vmin=-eta_max, vmax=eta_max, 
                             extent=domain_extent)
        plt.colorbar(sp[0], shrink=0.9)
        plt.axis('image')
        plt.title("$\eta{}$")
        
        ax[1] = plt.subplot(1, 3, 2)
        sp[1] = ax[1].imshow(hu, interpolation="none", origin='lower', 
                            cmap=plt.cm.coolwarm, vmin=-hu_max, vmax=hu_max, 
                            extent=domain_extent)
        plt.colorbar(sp[1], shrink=0.9)
        plt.axis('image')
        plt.title("$hu$")

        ax[2] = plt.subplot(1, 3, 3)
        sp[2] = ax[2].imshow(hv, interpolation="none", origin='lower', 
                             cmap=plt.cm.coolwarm, vmin=-hu_max, vmax=hu_max, 
                             extent=domain_extent)
        plt.colorbar(sp[2], shrink=0.9)
        plt.axis('image')
        plt.title("$hv$")
            
    else:        
        #Update plots
        fig.sca(ax[0])
        sp[0].set_data(eta)
        
        fig.sca(ax[1])
        sp[1].set_data(hu)
        
        fig.sca(ax[2])
        sp[2].set_data(hv)
        
    return ax, sp


def sim_animation(sim, T, anim_dt, hu_max=20.0, eta_max=0.5, comment=""):
    """
    Creates an animation of the simulator based on T frames, with frames sub_dt*sim.dt appart.
    """

    
    #Create figure and plot initial conditions
    fig = plt.figure(figsize=(14, 4))
    ax, sp = plotSolution(fig, sim, comment=comment,
                          eta_max=eta_max, hu_max=hu_max)
    #Helper function which simulates and plots the solution
    def animate(i):
        if (i>0):
            t = sim.step(anim_dt)
        else:
            t = 0.0

        plotSolution(fig, sim, comment=comment, ax=ax, sp=sp)
        print(".", end='')
        
    #Matplotlib for creating an animation
    anim = animation.FuncAnimation(fig, animate, range(T), interval=100)
    plt.close(fig)
    return anim

### Select url for initial conditions, domain, and simulate!

In [ ]:
source_url = 'https://thredds.met.no/thredds/dodsC/fou-hi/norkyst800m-1h/NorKyst-800m_ZDEPTHS_his.an.2019071600.nc'

casename = 'lofoten'

case_info = NetCDFInitialization.getCaseLocation(casename)
print(case_info["name"])

data_args_0 = NetCDFInitialization.getInitialConditions(source_url,case_info["x0"], case_info["x1"], case_info["y0"], case_info["y1"], download_data=False)
data_args_0 = NetCDFInitialization.removeMetadata(data_args_0)

data_args_1 = NetCDFInitialization.getInitialConditions(source_url,case_info["x0"], case_info["x1"], case_info["y0"], case_info["y1"], download_data=False)
data_args_1 = NetCDFInitialization.removeMetadata(data_args_1)

data_args_2 = NetCDFInitialization.getInitialConditions(source_url,case_info["x0"], case_info["x1"], case_info["y0"], case_info["y1"], download_data=False)
data_args_2 = NetCDFInitialization.removeMetadata(data_args_2)



In [ ]:
# Ensure that eta = hu = hv = 0.0
# Ensure all BCs are zero as well, 
#     --> except at the north
def stripDataArgs(data_args, bc_val_north=0.0):
    data_args["eta0"] *= 0.0
    data_args["hu0"] *= 0.0
    data_args["hv0"] *= 0.0
    data_args["wind_stress"] = WindStress.WindStress()

    data_args["boundary_conditions_data"].north.h *= 0.0
    data_args["boundary_conditions_data"].north.hu *= 0.0
    data_args["boundary_conditions_data"].north.hv *= 0.0
    data_args["boundary_conditions_data"].south.h *= 0.0
    data_args["boundary_conditions_data"].south.hu *= 0.0
    data_args["boundary_conditions_data"].south.hv *= 0.0
    data_args["boundary_conditions_data"].west.h *= 0.0
    data_args["boundary_conditions_data"].west.hu *= 0.0
    data_args["boundary_conditions_data"].west.hv *= 0.0
    data_args["boundary_conditions_data"].east.h *= 0.0
    data_args["boundary_conditions_data"].east.hu *= 0.0
    data_args["boundary_conditions_data"].east.hv *= 0.0
    
    data_args["boundary_conditions_data"].north.h += bc_val_north
    

stripDataArgs(data_args_0)
stripDataArgs(data_args_1,  1.0)
stripDataArgs(data_args_2, -1.0)




# Check lake at rest

In [ ]:
if "sim" in globals():
    sim.cleanUp()
    
sim = CDKLM16.CDKLM16(gpu_ctx1, dt=0.0, write_netcdf=False, **data_args_0)
T = 10
anim_dt = 3*60
sim_animation(sim, T, anim_dt, hu_max=50.0, eta_max=2, comment=case_info["name"])

# Check positive wave
 

In [ ]:
if "sim" in globals():
    sim.cleanUp()
    
sim = CDKLM16.CDKLM16(gpu_ctx1, dt=0.0, write_netcdf=False, **data_args_1)
T = 40
anim_dt = 3*60
sim_animation(sim, T, anim_dt, hu_max=50.0, eta_max=2, comment=case_info["name"])

# Check negative wave

In [ ]:
if "sim" in globals():
    sim.cleanUp()
    
sim = CDKLM16.CDKLM16(gpu_ctx1, dt=0.0, write_netcdf=False, **data_args_2)
T = 40
anim_dt = 3*60
sim_animation(sim, T, anim_dt, hu_max=50.0, eta_max=2, comment=case_info["name"])

# Step both other using same ctx

In [ ]:
if "sim_pos" in globals():
    sim_pos.cleanUp()
if "sim_neg" in globals():
    sim_neg.cleanUp()
    
sim_pos = CDKLM16.CDKLM16(gpu_ctx1, dt=0.0, write_netcdf=False, **data_args_1)
sim_neg = CDKLM16.CDKLM16(gpu_ctx1, dt=0.0, write_netcdf=False, **data_args_2)

T = 9*60
sim_pos.step(T)
fig = plt.figure(figsize=(15,4))
_, _ = plotSolution(fig, sim_pos, hu_max=50.0, eta_max=2, comment="pos")

sim_neg.step(T)
fig = plt.figure(figsize=(15,4))
_, _ = plotSolution(fig, sim_neg, hu_max=50.0, eta_max=2, comment="neg")

sim_pos.step(T)
fig = plt.figure(figsize=(15,4))
_, _ = plotSolution(fig, sim_pos, hu_max=50.0, eta_max=2, comment="pos")

sim_neg.step(T)
fig = plt.figure(figsize=(15,4))
_, _ = plotSolution(fig, sim_neg, hu_max=50.0, eta_max=2, comment="neg")

sim_neg.step(T)
fig = plt.figure(figsize=(15,4))
_, _ = plotSolution(fig, sim_neg, hu_max=50.0, eta_max=2, comment="neg")

sim_pos.step(T)
fig = plt.figure(figsize=(15,4))
_, _ = plotSolution(fig, sim_pos, hu_max=50.0, eta_max=2, comment="pos")




# Step both using separate ctx

In [ ]:
if "sim_pos" in globals():
    sim_pos.cleanUp()
if "sim_neg" in globals():
    sim_neg.cleanUp()
    
sim_pos = CDKLM16.CDKLM16(gpu_ctx1, dt=0.0, write_netcdf=False, **data_args_1)
sim_neg = CDKLM16.CDKLM16(gpu_ctx2, dt=0.0, write_netcdf=False, **data_args_2)

T = 9*60
sim_pos.step(T)
fig = plt.figure(figsize=(15,4))
_, _ = plotSolution(fig, sim_pos, hu_max=50.0, eta_max=2, comment="pos")

sim_neg.step(T)
fig = plt.figure(figsize=(15,4))
_, _ = plotSolution(fig, sim_neg, hu_max=50.0, eta_max=2, comment="neg")

sim_pos.step(T)
fig = plt.figure(figsize=(15,4))
_, _ = plotSolution(fig, sim_pos, hu_max=50.0, eta_max=2, comment="pos")

sim_neg.step(T)
fig = plt.figure(figsize=(15,4))
_, _ = plotSolution(fig, sim_neg, hu_max=50.0, eta_max=2, comment="neg")

sim_neg.step(T)
fig = plt.figure(figsize=(15,4))
_, _ = plotSolution(fig, sim_neg, hu_max=50.0, eta_max=2, comment="neg")

sim_pos.step(T)
fig = plt.figure(figsize=(15,4))
_, _ = plotSolution(fig, sim_pos, hu_max=50.0, eta_max=2, comment="pos")

